In [1]:
import numpy as np 
import pandas as pd

In [2]:
import mne
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold

**Load data from csv files.**

In [3]:
df_A = pd.read_csv("C:/SEAN/GA Tech Academics/Grad\Fall 2024/ML CSE 8803/ML-CSE-8803-Team-2/Classification/Stress/Datasets/F_Relax_A_feature.csv", header = 0)
df_M = pd.read_csv("C:/SEAN/GA Tech Academics/Grad\Fall 2024/ML CSE 8803/ML-CSE-8803-Team-2/Classification/Stress/Datasets/F_Relax_M_feature.csv", header = 0)
df_S = pd.read_csv("C:/SEAN/GA Tech Academics/Grad\Fall 2024/ML CSE 8803/ML-CSE-8803-Team-2/Classification/Stress/Datasets/F_Relax_S_feature.csv", header = 0)

<>:1: SyntaxWarning: invalid escape sequence '\F'
<>:2: SyntaxWarning: invalid escape sequence '\F'
<>:3: SyntaxWarning: invalid escape sequence '\F'
<>:1: SyntaxWarning: invalid escape sequence '\F'
<>:2: SyntaxWarning: invalid escape sequence '\F'
<>:3: SyntaxWarning: invalid escape sequence '\F'
C:\Users\sean\AppData\Local\Temp\ipykernel_24116\3915618476.py:1: SyntaxWarning: invalid escape sequence '\F'
  df_A = pd.read_csv("C:/SEAN/GA Tech Academics/Grad\Fall 2024/ML CSE 8803/ML-CSE-8803-Team-2/Classification/Stress/Datasets/F_Relax_A_feature.csv", header = 0)
C:\Users\sean\AppData\Local\Temp\ipykernel_24116\3915618476.py:2: SyntaxWarning: invalid escape sequence '\F'
  df_M = pd.read_csv("C:/SEAN/GA Tech Academics/Grad\Fall 2024/ML CSE 8803/ML-CSE-8803-Team-2/Classification/Stress/Datasets/F_Relax_M_feature.csv", header = 0)
C:\Users\sean\AppData\Local\Temp\ipykernel_24116\3915618476.py:3: SyntaxWarning: invalid escape sequence '\F'
  df_S = pd.read_csv("C:/SEAN/GA Tech Academics/

Fill in missing data with the mode of its column.

In [4]:
df_A = df_A.fillna(df_A.mode().iloc[0])
df_A = df_A.iloc[:, 1:] # Drop the leftmost column because it is just an ID number.
df_M = df_M.fillna(df_M.mode().iloc[0])
df_M = df_M.iloc[:, 1:]
df_S = df_S.fillna(df_S.mode().iloc[0])
df_S = df_S.iloc[:, 1:]

Preprocess the data (There are 32 electrodes and 450 columns total, so 14 columns per electrode).

PCA: https://www.geeksforgeeks.org/implementing-pca-in-python-with-scikit-learn/

FFS: https://www.analyticsvidhya.com/blog/2021/04/forward-feature-selection-and-its-implementation/


In [5]:
def preprocess(data: pd.DataFrame) -> pd.DataFrame: # COME BACK LATER
    data_arr = (data.to_numpy()).T
    channel_names = list(df_A.columns)  
    channel_types = ['eeg'] * len(channel_names) 
    info = mne.create_info(ch_names = channel_names, ch_types = channel_types)
    raw_arr = mne.io.RawArray(data_arr, info)
    raw_arr.filter(1, 40)
    
    # Perform ICA 
    ica = mne.preprocessing.ICA(n_components = 3)
    ica.fit(raw_arr)
    ica.exclude = ica.find_bads_eog(raw_arr)[0]
    raw_ica_cleaned = ica.apply(raw_arr.copy())
    
    # Convert cleaned data back into pandas DataFrame
    processed_data = (raw_ica_cleaned.get_data()).T
    df_processed = pd.DataFrame(processed_data, columns = channel_names)
    raw_ica_cleaned.plot()
    return df_processed
    

In [5]:
df_A.head()

,0.psd_delta,0.psd_theta,0.psd_alpha,0.psd_beta,0.psd_gamma,0.theta_beta_ratio,0.theta_alpha_ratio,0.alpha_beta_ratio,0.spectral_entropy_value,0.sample_entropy_value,...,31.theta_beta_ratio,31.theta_alpha_ratio,31.alpha_beta_ratio,31.spectral_entropy_value,31.sample_entropy_value,31.delta_rel_power,31.theta_rel_power,31.alpha_rel_power,31.beta_rel_power,Stress_level
0,14850.830823,14923.215094,7910.065985,6950.560915,3047.582095,2.147052,1.886611,1.138047,4.911304,2.639057,...,1.916498,1.553704,1.233503,4.733905,1.609438,0.135955,0.134893,0.206889,0.308508,1
1,14838.280527,18650.065251,7921.853628,8087.625726,2855.607519,2.306000,2.354255,0.979503,4.778889,1.098612,...,1.576940,1.477379,1.067390,4.884945,0.000000,0.152942,0.047734,0.071766,0.553523,1
2,7143.715839,16263.647994,6245.775968,4200.151132,2838.155117,3.872158,2.603944,1.487036,4.455605,0.000000,...,1.827205,1.172337,1.558600,4.681201,1.897120,0.225457,0.153719,0.120138,0.303591,1
3,7143.570136,12413.634950,7677.763201,4200.428316,2764.924613,2.955326,1.616830,1.827852,4.594508,0.000000,...,0.942636,1.648822,0.571703,4.781241,0.000000,0.166542,0.045211,0.036688,0.357148,1
4,19573.432251,18451.900407,7663.019348,6509.398324,2423.619039,2.834655,2.407915,1.177224,4.605586,1.386294,...,4.879218,3.489728,1.398166,4.639766,2.397895,0.168762,0.150286,0.064078,0.482854,1


In [6]:
start_column_index = 0
for i in range(32):
    df_subset = df_A.iloc[:, start_column_index: start_column_index + 14]
    
    # PCA for dimensionality reduction:
    
    # First scale the data.
    scaler = StandardScaler()
    scaler.fit(df_subset)
    scaled_df_subset = scaler.transform(df_subset)
    
    # Now apply PCA on the scaled data.
    pca = PCA(n_components = 2)
    pca.fit(scaled_df_subset)
    pca_result = pca.transform(scaled_df_subset)
    print(pca_result.shape)
    
    start_column_index += 14

(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)
(6000, 2)


After performing dimensionality reduction, visualize the remaining features.